# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')

token = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
from itertools import accumulate


In [5]:
length_to_split = [100, 100, 100, 100, 100]

In [6]:
symbol_groups = [stocks['Ticker'][x - y: x] for x, y in zip(
          accumulate(length_to_split), length_to_split)]

In [7]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [8]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [9]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [17]:
final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={token}'
    data = requests.get(batch_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
        pd.Series(
                    [
                        symbol, 
                        data[symbol]['price'],
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A'
                    ], index = my_columns
        )
        , ignore_index = True)
        
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,125.80,-0.179838,N/A
1,AAL,14.60,-0.333471,N/A
2,AAP,184.80,-0.126408,N/A
3,AAPL,154.36,0.019993,N/A
4,ABBV,158.96,0.378708,N/A
...,...,...,...,...
496,YUM,120.32,0.006349,N/A
497,ZBH,109.68,-0.336736,N/A
498,ZBRA,302.38,-0.457581,N/A
499,ZION,53.28,-0.024787,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [18]:
final_df.sort_values('One-Year Price Return', ascending=False, inplace=True)

final_df = final_df[:50]

final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
273,LB,81.84,2.35259,N/A
147,DVN,56.07,1.064988,N/A
355,OXY,62.09,1.055817,N/A
298,MCK,333.80,0.762332,N/A
89,CF,87.00,0.751857,N/A
42,APA,36.55,0.724384,N/A
138,DLTR,169.38,0.69093,N/A
315,MRO,22.66,0.675242,N/A
225,HRB,38.96,0.631858,N/A
463,VAR,181.06,0.582425,N/A


In [19]:
final_df.reset_index(inplace=True)

final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,81.84,2.35259,N/A
1,147,DVN,56.07,1.064988,N/A
2,355,OXY,62.09,1.055817,N/A
3,298,MCK,333.80,0.762332,N/A
4,89,CF,87.00,0.751857,N/A
5,42,APA,36.55,0.724384,N/A
6,138,DLTR,169.38,0.69093,N/A
7,315,MRO,22.66,0.675242,N/A
8,225,HRB,38.96,0.631858,N/A
9,463,VAR,181.06,0.582425,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [24]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")
    
    try:
        portfolio_size float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [26]:
size_df = len(final_df.index)

position_size = float(portfolio_size)/size_df

In [27]:
for i in range(0, size_df):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])
    
final_df

C:\Users\Aditi\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,81.84,2.35259,244
1,147,DVN,56.07,1.064988,356
2,355,OXY,62.09,1.055817,322
3,298,MCK,333.80,0.762332,59
4,89,CF,87.00,0.751857,229
5,42,APA,36.55,0.724384,547
6,138,DLTR,169.38,0.69093,118
7,315,MRO,22.66,0.675242,882
8,225,HRB,38.96,0.631858,513
9,463,VAR,181.06,0.582425,110


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [28]:
hqm_cols = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy', 
    'One-Year Price Return', 
    'One-Year Return Percentile', 
    'Six-Month Price Return', 
    'Six-Month Return Percentile', 
    'Three-Month Price Return', 
    'Three-Month Return Percentile', 
    'One-Month Price Return', 
    'One-Month Return Percentile', 
]

hqm_df = pd.DataFrame(columns = hqm_cols)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile


In [30]:
for symbol_string in symbol_strings:
    batch_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={token}'
    data = requests.get(batch_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
        pd.Series(
                    [
                        symbol, 
    data[symbol]['price'], 
    'N/A', 
    data[symbol]['stats']['year1ChangePercent'] ,
    'N/A', 
    data[symbol]['stats']['month6ChangePercent'] , 
    'N/A', 
    data[symbol]['stats']['month3ChangePercent'] , 
    'N/A', 
    data[symbol]['stats']['month1ChangePercent'] , 
    'N/A' 
                   ], index = hqm_cols
        )
        , ignore_index = True)

In [31]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,126.40,N/A,-0.182171,N/A,-0.158218,N/A,-0.09118,N/A,0.018405,N/A
1,AAL,13.97,N/A,-0.338932,N/A,-0.256856,N/A,-0.159766,N/A,-0.052727,N/A
2,AAP,185.80,N/A,-0.124501,N/A,-0.214524,N/A,-0.185861,N/A,0.028963,N/A
3,AAPL,147.64,N/A,0.020025,N/A,-0.150519,N/A,-0.139025,N/A,0.074069,N/A
4,ABBV,159.36,N/A,0.381838,N/A,0.14699,N/A,-0.120351,N/A,0.070099,N/A
...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,119.79,N/A,0.006393,N/A,-0.11607,N/A,-0.011349,N/A,0.02556,N/A
497,ZBH,107.97,N/A,-0.351594,N/A,-0.166733,N/A,-0.18619,N/A,-0.069821,N/A
498,ZBRA,307.92,N/A,-0.470797,N/A,-0.458127,N/A,-0.283575,N/A,-0.019441,N/A
499,ZION,52.49,N/A,-0.024311,N/A,-0.273185,N/A,-0.210753,N/A,-0.034691,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [32]:
time_periods = [
    'One-Year', 
    'Six-Month', 
    'Three-Month',
    'One-Month'
]

In [48]:
hqm_df = hqm_df.dropna()

In [49]:
hqm_df.shape

(497, 11)

In [50]:
hqm_df.isnull().sum()

Ticker                           0
Price                            0
Number of Shares to Buy          0
One-Year Price Return            0
One-Year Return Percentile       0
Six-Month Price Return           0
Six-Month Return Percentile      0
Three-Month Price Return         0
Three-Month Return Percentile    0
One-Month Price Return           0
One-Month Return Percentile      0
dtype: int64

In [51]:
#for row in hqm_df.index:
#    for time_period in time_periods:
#        hqm_df.loc[row, f'{time_period} Return Percentile'] = hqm_dataframe.loc[row, f'{time_period} Price Return'])/100


for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        
        hqm_df.loc[row, percentile_col] = stats.percentileofscore(hqm_df[change_col], 
                                                                  (hqm_df.loc[row, change_col])/100)    


C:\Users\Aditi\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [52]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,126.40,N/A,-0.182171,63.380282,-0.158218,76.257545,-0.09118,85.915493,0.018405,60.965795
1,AAL,13.97,N/A,-0.338932,63.380282,-0.256856,76.257545,-0.159766,85.714286,-0.052727,58.752515
2,AAP,185.80,N/A,-0.124501,63.380282,-0.214524,76.257545,-0.185861,85.714286,0.028963,60.965795
3,AAPL,147.64,N/A,0.020025,65.593561,-0.150519,76.257545,-0.139025,85.714286,0.074069,61.368209
4,ABBV,159.36,N/A,0.381838,66.39839,0.14699,79.074447,-0.120351,85.915493,0.070099,61.167002
...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,119.79,N/A,0.006393,65.593561,-0.11607,76.458753,-0.011349,86.1167,0.02556,60.965795
497,ZBH,107.97,N/A,-0.351594,63.380282,-0.166733,76.257545,-0.18619,85.714286,-0.069821,58.752515
498,ZBRA,307.92,N/A,-0.470797,63.380282,-0.458127,76.056338,-0.283575,85.714286,-0.019441,59.15493
499,ZION,52.49,N/A,-0.024311,63.782696,-0.273185,76.056338,-0.210753,85.714286,-0.034691,59.15493


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [54]:
hqm_df['HQM Score'] = ''

hqm_df.isnull().sum()

C:\Users\Aditi\AppData\Local\Temp\ipykernel_20584\955013483.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_df['HQM Score'] = ''


Ticker                           0
Price                            0
Number of Shares to Buy          0
One-Year Price Return            0
One-Year Return Percentile       0
Six-Month Price Return           0
Six-Month Return Percentile      0
Three-Month Price Return         0
Three-Month Return Percentile    0
One-Month Price Return           0
One-Month Return Percentile      0
HQM Score                        0
dtype: int64

In [57]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_df

C:\Users\Aditi\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.40,N/A,-0.182171,63.380282,-0.158218,76.257545,-0.09118,85.915493,0.018405,60.965795,71.629779
1,AAL,13.97,N/A,-0.338932,63.380282,-0.256856,76.257545,-0.159766,85.714286,-0.052727,58.752515,71.026157
2,AAP,185.80,N/A,-0.124501,63.380282,-0.214524,76.257545,-0.185861,85.714286,0.028963,60.965795,71.579477
3,AAPL,147.64,N/A,0.020025,65.593561,-0.150519,76.257545,-0.139025,85.714286,0.074069,61.368209,72.2334
4,ABBV,159.36,N/A,0.381838,66.39839,0.14699,79.074447,-0.120351,85.915493,0.070099,61.167002,73.138833
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,119.79,N/A,0.006393,65.593561,-0.11607,76.458753,-0.011349,86.1167,0.02556,60.965795,72.283702
497,ZBH,107.97,N/A,-0.351594,63.380282,-0.166733,76.257545,-0.18619,85.714286,-0.069821,58.752515,71.026157
498,ZBRA,307.92,N/A,-0.470797,63.380282,-0.458127,76.056338,-0.283575,85.714286,-0.019441,59.15493,71.076459
499,ZION,52.49,N/A,-0.024311,63.782696,-0.273185,76.056338,-0.210753,85.714286,-0.034691,59.15493,71.177062


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [60]:
hqm_df.sort_values('HQM Score', ascending = False, inplace = True)

hqm_df = hqm_df[:50]

hqm_df

C:\Users\Aditi\AppData\Local\Temp\ipykernel_20584\2718390849.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_df.sort_values('HQM Score', ascending = False, inplace = True)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
273,LB,81.63,N/A,2.320948,69.416499,0.829825,80.080483,0.225455,88.732394,0.080447,61.368209,74.899396
107,COG,23.03,N/A,0.343317,66.39839,0.184902,79.074447,0.257073,88.732394,0.261018,62.977867,74.295775
225,HRB,37.33,N/A,0.638918,66.599598,0.671717,79.879276,0.416392,88.732394,0.053073,61.167002,74.094567
138,DLTR,175.20,N/A,0.693579,67.002012,0.240211,79.074447,0.027962,88.128773,0.07491,61.368209,73.89336
298,MCK,336.00,N/A,0.798843,67.203219,0.324354,79.074447,0.017159,87.927565,0.059478,61.167002,73.843058
470,VRTX,300.68,N/A,0.48736,66.39839,0.321143,79.074447,0.051527,88.128773,0.160599,61.770624,73.843058
199,GIS,77.96,N/A,0.309514,66.39839,0.117998,79.074447,0.086037,88.128773,0.115967,61.569416,73.792757
281,LLY,337.77,N/A,0.428277,66.39839,0.290488,79.074447,0.066157,88.128773,0.117533,61.569416,73.792757
310,MNST,100.87,N/A,0.08337,65.593561,0.053704,78.873239,0.198149,88.732394,0.12105,61.569416,73.692153
94,CI,280.32,N/A,0.215187,65.995976,0.20567,79.074447,0.088799,88.128773,0.102912,61.569416,73.692153


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [67]:
portfolio_input()

Enter the value of your portfolio:10000000


In [74]:
position_size = float(portfolio_size)/size_df

for i in range(0, size_df):
    if(hqm_df.loc[i, 'Price'] == 0):
        print(hqm_df.loc[i, 'Ticker'])
    #hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])
    
#hqm_df

KeyError: 0

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: